In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train =pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
test = pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')
df_test = test['comment_text']
df = train[['target','comment_text']]

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
X = df['comment_text']
y = df['target']
y = np.where(y>=0.5,1.,0.)
x_train, x_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, random_state=42)

In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
unique, counts

In [ ]:
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(x_train.values.reshape(-1,1), y_train)
print(sorted(Counter(y_resampled).items()))

In [ ]:
X_resampled = X_resampled.reshape(-1)

In [ ]:
X_resampled.shape

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

vec = CountVectorizer()
clf = LogisticRegression()
pipe = make_pipeline(vec, clf)
pipe.fit(X_resampled, y_resampled);

In [ ]:
from sklearn import metrics

def print_report(pipe, x_test, y_test):
    y_pred = pipe.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

print_report(pipe, x_test, y_test)

In [ ]:
import eli5
eli5.show_weights(clf, vec=vec, top=20)

In [ ]:
for _, row in df.sample(5).iterrows():
    print(f"true label: {row['target']}")
    display(eli5.show_prediction(clf, row['comment_text'], vec=vec,))
    print("--"*50)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
pipe_rf = make_pipeline(vec, rf)
pipe_rf.fit(X_resampled, y_resampled)

In [ ]:
print_report(pipe_rf, x_test, y_test)

In [ ]:
vec2 = TfidfVectorizer(analyzer='char_wb', ngram_range=(3, 5), min_df=.01, max_df=.5)
clf2 = LinearSVC()
pipe_tfidf = make_pipeline(vec2, clf2)
pipe_tfidf.fit(X_resampled, y_resampled);

In [ ]:
print_report(pipe_tfidf, x_test, y_test)

In [ ]:
eli5.show_weights(clf2, vec=vec2, top=20)

In [ ]:
for _, row in df.sample(5).iterrows():
    print(f"true label: {row['target']}")
    display(eli5.show_prediction(clf2, row['comment_text'], vec=vec2,))
    print("--"*50)

In [ ]:
pipe_tfidf_rf = make_pipeline(vec2, rf)
pipe_tfidf_rf.fit(X_resampled, y_resampled);

In [ ]:
print_report(pipe_tfidf_rf, x_test, y_test)

In [ ]:
y_pred= pipe.predict(df_test)

In [ ]:
y_pred = np.where(y_pred>=0.5,1.,0.)
sub_df = pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv')
sub_df['prediction'] = y_pred
sub_df.to_csv('submission.csv',index = False)

In [ ]:
y_pred_2= pipe_tfidf.predict(df_test)

In [ ]:
y_pred_2 = np.where(y_pred_2>=0.5,1.,0.)
sub_df = pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv')
sub_df['prediction'] = y_pred_2
sub_df.to_csv('submission_2.csv',index = False)